# Importing Libraries

In [ ]:
import re
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf

# Removing characters with <>

In [ ]:
del_chr = r'<.*?>'

infile = open('train-data.dat', 'r')
lines = infile.readlines()
infile.close()
     
outfile = open('train-data_new.dat', 'w')

for str in lines:
    new_str = re.sub(del_chr, '', str)
    outfile.writelines(new_str)
    
outfile.close()

In [ ]:
del_chr = r'<.*?>'

infile = open('test-data.dat', 'r')
lines = infile.readlines()
infile.close()
     
outfile = open('test-data_new.dat', 'w')

for str in lines:
    new_str = re.sub(del_chr, '', str)
    outfile.writelines(new_str)
    
outfile.close()

# Creating the BoW model

In [ ]:
word2count1 = {}

dataset = open('train-data_new.dat', 'r')
for data in dataset:
    words = nltk.word_tokenize(data)
    for word in words:
        if word not in word2count1.keys():
            word2count1[word] = 1
        else:
            word2count1[word] += 1
dataset.close()

In [ ]:
word2count2 = {}

dataset = open('test-data_new.dat', 'r')
for data in dataset:
    words = nltk.word_tokenize(data)
    for word in words:
        if word not in word2count2.keys():
            word2count2[word] = 1
        else:
            word2count2[word] += 1
dataset.close()

In [ ]:
len(word2count1), len(word2count2)

In [ ]:
infile = open('train-data_new.dat', 'r')
lines1 = infile.readlines()
infile.close()
     
#outfile = open('train-data_new_tokenized.dat', 'w')

for str in lines1:
    new_str = word_tokenize(str)
    print(new_str)
    #outfile.writelines(new_str)
    
#outfile.close()

In [ ]:
infile = open('test-data_new.dat', 'r')
lines2 = infile.readlines()
infile.close()
     
#outfile = open('train-data_new_tokenized.dat', 'w')

for str in lines2:
    new_str = word_tokenize(str)
    print(new_str)
    #outfile.writelines(new_str)
    
#outfile.close()

## Model 1

In [ ]:
model1 = Tokenizer()
model1.fit_on_texts(lines1)

In [ ]:
rep1 = model1.texts_to_matrix(lines1, mode='count')

In [ ]:
#removing one extra column
rep1 = np.delete(rep1,0,1)
rep1

In [ ]:
# Get the maximum element to chech for outliers
maxElement = np.amax(rep1)
maxElement

## Model 2

In [ ]:
model2 = Tokenizer()
model2.fit_on_texts(lines2)

In [ ]:
rep2 = model2.texts_to_matrix(lines2, mode='count')

In [ ]:
#removing one extra column
rep2 = np.delete(rep2,0,1)
rep2

In [ ]:
maxElement = np.amax(rep2)
maxElement

In [ ]:
list(model1.word_index.keys())

In [ ]:
list(model2.word_index.keys())

### Check the vocabularies' lengths

In [ ]:
train_vocabulary = list(model1.word_index.keys())

In [ ]:
len(train_vocabulary)

In [ ]:
test_vocabulary = list(model2.word_index.keys())

In [ ]:
len(test_vocabulary)

### Find missing words from test vocabulary

In [ ]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match
       

list_a = train_vocabulary
list_b = test_vocabulary

non_match = non_match_elements(list_a, list_b)
print("No match elements: ", non_match)
print("Number of no match elements: ", len(non_match))

# Preprocessing Data

In [ ]:
#Making a dataframe out of train vocabulary
df_train = pd.DataFrame(rep1, columns = train_vocabulary)
df_train

In [ ]:
#Sorting it
df_train = df_train.sort_index(axis=1)

In [ ]:
df_train

In [ ]:
#Making a dataframe out of test vocabulary
df_test = pd.DataFrame(rep2, columns = test_vocabulary)
df_test

In [ ]:
#Sorting it
df_test = df_test.sort_index(axis=1)

In [ ]:
df_test

In [ ]:
#Making a dataframe out of the missing words from test vocabulary
df_temp = pd.DataFrame(columns = non_match)
df_temp

In [ ]:
#Combining test vocabulary and missing words to match the size of train vocabulary
df_test_final = df_test.append(df_temp)
df_test_final

In [ ]:
#Change null values with zeros
df_test_final = df_test_final.fillna(0)
df_test_final

In [ ]:
df_test_final = df_test_final.astype(float)

In [ ]:
df_test_final = df_test_final.sort_index(axis=1)

In [ ]:
df_test_final, df_train

In [ ]:
file = "train-label.dat"
colnames = []

for i in range(20):
    colnames.append(i)

df_train_label = pd.read_csv(file,sep=' ', names=colnames)
df_train_label

In [ ]:
file = "test-label.dat"
colnames = []

for i in range(20):
    colnames.append(i)

df_test_label = pd.read_csv(file,sep=' ', names=colnames)
df_test_label

## Turn dataframes into csvs

In [ ]:
# df_train.to_csv('train-data.csv')
# df_test_final.to_csv('test-data.csv')
# df_train_label.to_csv('train-label.csv')
# df_test_label.to_csv('test-label.csv')

### Centering

In [ ]:
df_train.mean(axis=1)

In [ ]:
df_train_centered = df_train.copy()
df_train_centered = df_train_centered.sub(df_train_centered.mean(axis=1), axis=0)
df_train_centered, df_train_centered.mean(axis=1)

### Normalization

In [ ]:
# copy the data
df_train_normalized = df_train.copy()
  
# apply normalization techniques
for column in df_train_normalized.columns:
    df_train_normalized[column] = (df_train_normalized[column] - df_train_normalized[column].min()) / (df_train_normalized[column].max() - df_train_normalized[column].min())    
  
df_train_normalized['2287']

### Standardization

#### 1st Way

In [ ]:
# copy the data
df_train_standardized = df_train.copy()

# apply standardization techniques
df_train_standardized = (df_train_standardized-df_train_standardized.mean())/df_train_standardized.std()
df_train_standardized

In [ ]:
df_train_standardized.std()

In [ ]:
X = df_train.copy()
scale_X = StandardScaler().fit(X)
X_norm = scale_X.transform(X)
X_norm
df_standardized = pd.DataFrame(X_norm, columns = df_train.columns)
df_standardized
df_standardized.std()


#### 2nd Way

In [ ]:
X = df_train.copy()
scale_X = StandardScaler().fit(X)
X_norm = scale_X.transform(X)

In [ ]:
X_norm

In [ ]:
df_standardized = pd.DataFrame(X_norm, columns = df_train.columns)
df_standardized

In [ ]:
df_standardized.std()